In [56]:
import numpy as np
from numpy import array
import matplotlib.pyplot as plt

from itertools import combinations

cumulative_percentages = array([0.000154,0.0015,0.0256,0.17,0.367,0.76,2.87,7.62,49.9][::-1])/100
names = ["Nothing", "Pair", "Two pair", "Three of a kind", "Straight", "Flush", "Full house", "Four of a kind", "Straight flush", "Royal flush"]


In [111]:
def get_shuffled_deck():
    arr = array([array([np.linspace(0,12,13).astype(int),(np.ones(13)*i).astype(int)]).T for i in range(4)]).reshape(52,2)
    
    np.random.shuffle(arr)
    return arr

def check_for_pairs(cards):
    _, uniques = np.unique(cards[:,0],return_counts=True)
    if 4 in uniques:
        return "FourOfAKind"
    elif 3 in uniques:
        if 2 in uniques:
            return "FullHouse"
        return "ThreeOfAKind"
    elif 2 in uniques:
        if sum(uniques == 2)==2:
            return "TwoPairs"
        return "Pair"
    return None

def check_for_straights(cards):
    sort = np.sort(cards[:,0])
    if sum((sort[1:] - sort[:-1]) != 1) == 0:
        if sort[-1] == 12:
            return "RoyalStraight"
        return "Straight"
    return None

def check_for_flush(cards):
    _, unique_suits = np.unique(cards[:,1], return_counts = True)

    if 5 in unique_suits:
        return "Flush"
    return None

def get_hand_value(cards):
    if cards.shape[0] != 5:
        print("Total cards are",cards.shape[0])
        return

    cv = get_combination_value(cards)

    high = np.max(cards[:,0])

    return 13*cv + high

def get_combination_value(cards):
    """
    0 Nothing
    1 Pair
    2 Two pair
    3 Three of a kind
    4 Straight
    5 Flush
    6 Full house
    7 Four of a kind
    8 Straight flush
    9 Royal flush
    """

    pairs = check_for_pairs(cards)
    straight = check_for_straights(cards)
    flush = check_for_flush(cards)
    
    
    if flush == "Flush" and straight == "RoyalStraight":
        return 9
    if flush == "Flush" and straight == "Straight":
        return 8
    if pairs == "FourOfAKind":
        return 7
    if pairs == "FullHouse":
        return 6
    if flush == "Flush":
        return 5
    if straight == "Straight":
        return 4
    if pairs == "ThreeOfAKind":
        return 3
    if pairs == "TwoPairs":
        return 2
    if pairs == "Pair":
        return 1
    return 0
   

class Game():
    def __init__(self):
        self.hand = array([])
        self.opponent_hand = array([])
        self.open_cards = array([])

    def start_game(self):
        self.deck = get_shuffled_deck()

    def draw_hand(self):
        self.hand = self.deck[:2]
        self.deck = self.deck[2:]

    def get_hand_value(self,hand):
        return get_hand_value(np.concatenate((self.open_cards,hand)))

    def draw_opponent_hand(self):
        self.opponent_hand = self.deck[:2]
        self.deck = self.deck[2:]
    
    def show_card(self):
        self.open_cards = np.append(self.open_cards, self.deck[0]).reshape(-1,2).astype(int)
        self.deck = self.deck[1:]

    def calc_probability_for_higher(self):
        hand = self.hand.copy()
        open_cards = self.open_cards.copy()

        if open_cards.shape[0] < 2:
            hand_val = get_combination_value(np.concatenate((hand,open_cards.reshape(-1,2))))
            if hand_val == 0:
                return (1 - np.max(hand[:,0])/13) * 0.5 + 0.5
            return cumulative_percentages[hand_val]

        #Make Dynamic
        trydeck = get_shuffled_deck()

        removes = []
        for i, c in enumerate(trydeck):
            if all(c == hand[0]) or all(c == hand[1]) or any([all(c == oc) for oc in open_cards]):
                removes.append(i)
                
        trydeck = np.delete(trydeck, removes, axis = 0)

        hands = []
        for ha in combinations(range(trydeck.shape[0]),5-open_cards.shape[0]):
            opp_cards = array([trydeck[h] for h in ha])
            open = np.concatenate((open_cards,opp_cards[:(3-open_cards.shape[0])]))
            tryhand = np.concatenate((open_cards, opp_cards))

            opp_val = get_hand_value(tryhand) 

            p = (np.concatenate((open,hand)))
            # print(open.shape,open_cards.shape,hand.shape)
            player_val = get_hand_value(p) 

            hands.append(opp_val>player_val)

        return sum(hands)/len(hands)   

    def play(self):
        self.start_game()
        self.draw_hand()
        self.draw_opponent_hand()

        val = 0

        
        for i in range(3):
            val += np.random.random()*50/6+50/6
            pv = (1 - self.calc_probability_for_higher())*100
            if val > pv:
                print(f"Fold from bet {val} against a max of {pv}")
                break
            else:
                print(f"Player stays with a bet of {val:.3} against a max of {pv:.3}")

                self.show_card()
            print("Player hand:\n",self.hand,"\nOpponent hand:\n",self.opponent_hand,"\nOpen Cards:\n",self.open_cards,"\n")

        if self.open_cards.shape[0] == 3:
            pv = self.get_hand_value(self.hand)
            ov = self.get_hand_value(self.opponent_hand)

            if pv > ov:
                print(f"Player won!")
            else:
                print(f"Player lost :(")


            print(f"Player had {names[int((pv-pv%13)/13)]} with a high {pv%13}")
            print(f"Opponent had {names[int((ov-ov%13)/13)]} with a high {ov%13}")

# deck = get_shuffled_deck()

# for i in range(0,4):
#     hand, open = deck[:2], deck[2:(2+i)]
#     pro = calc_probability_for_higher(hand,open)
#     print(f"Hand: \n{hand[:2]}\nOpen: \n{open}\nProb: {pro*100:.3}\n")


game = Game()

game.play()

Player stays with a bet of 16.1 against a max of 30.8
Player hand:
 [[8 3]
 [5 0]] 
Opponent hand:
 [[ 2  2]
 [10  2]] 
Open Cards:
 [[5 3]] 

Player stays with a bet of 32.1 against a max of 92.4
Player hand:
 [[8 3]
 [5 0]] 
Opponent hand:
 [[ 2  2]
 [10  2]] 
Open Cards:
 [[5 3]
 [3 1]] 

Player stays with a bet of 41.3 against a max of 84.2
Player hand:
 [[8 3]
 [5 0]] 
Opponent hand:
 [[ 2  2]
 [10  2]] 
Open Cards:
 [[5 3]
 [3 1]
 [9 3]] 

Player won!
Player had Pair with a high 9
Opponent had Nothing with a high 10


In [ ]:
def print_distribution(N = 3*10000):
    vals = np.empty(N*8)
    for i in range(N):
            
        deck = get_shuffled_deck()
        for k in range(8):
            val = get_hand_value(deck[k*5:(k+1)*5],array([[],[]]).T)
            vals[i*8+k] = val
            
    N = 3*10000
    vals = np.empty(N*8)
    for i in range(N):
            
        deck = get_shuffled_deck()
        for k in range(8):
            val = get_hand_value(deck[k*5:(k+1)*5],array([[],[]]).T)
            vals[i*8+k] = val

    t = np.unique(vals,return_counts = True)

    for a,b in zip(*t):
        print(a,b/sum(t[1])*100)